In [151]:
# Import required libraries and dependencies
import os
os.environ["OMP_NUM_THREADS"] = "1"
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler



In [152]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384
binancecoin,2.10423,12.85511,6.80688,0.05865,36.33486,155.61937,69.69195
chainlink,-0.23935,20.69459,9.30098,-11.21747,-43.69522,403.22917,325.13186
cardano,0.00322,13.99302,5.55476,10.10553,-22.84776,264.51418,156.09756
litecoin,-0.06341,6.60221,7.28931,1.21662,-17.23960,27.49919,-12.66408


In [153]:
# Generate summary statistics
df_market_data.describe()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
count,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000,41.000000
mean,-0.269686,4.497147,0.185787,1.545693,-0.094119,236.537432,347.667956
std,2.694793,6.375218,8.376939,26.344218,47.365803,435.225304,1247.842884
min,-13.527860,-6.094560,-18.158900,-34.705480,-44.822480,-0.392100,-17.567530
25%,-0.608970,0.047260,-5.026620,-10.438470,-25.907990,21.660420,0.406170
50%,-0.063410,3.296410,0.109740,-0.042370,-7.544550,83.905200,69.691950
75%,0.612090,7.602780,5.510740,4.578130,0.657260,216.177610,168.372510
max,4.840330,20.694590,24.239190,140.795700,223.064370,2227.927820,7852.089700


In [154]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

:NdOverlay   [Variable]
   :Curve   [coin_id]   (value)

---

### Prepare the Data

In [108]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()

In [155]:
# Create a DataFrame with the scaled data
scaled_data = scaler.fit_transform(df_market_data)

# Copy the crypto names from the original data
crypto_names = df_market_data.index

# Set the coinid column as index
scaled_df = pd.DataFrame(scaled_data, columns=df_market_data.columns, index=crypto_names)

# Display sample data
scaled_df.head()

,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y
coin_id,,,,,,,
bitcoin,0.508529,0.493193,0.772200,0.235460,-0.067495,-0.355953,-0.251637
ethereum,0.185446,0.934445,0.558692,-0.054341,-0.273483,-0.115759,-0.199352
tether,0.021774,-0.706337,-0.021680,-0.061030,0.008005,-0.550247,-0.282061
ripple,-0.040764,-0.810928,0.249458,-0.050388,-0.373164,-0.458259,-0.295546
bitcoin-cash,1.193036,2.000959,1.760610,0.545842,-0.291203,-0.499848,-0.270317


---

### Find the Best Value for k Using the Original Data.

In [156]:
# Create a list with the number of k-values from 1 to 11
k_values = list(range(1, 12))

In [157]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # 2. Fit the model to the data using `df_market_data`
    kmeans.fit(df_market_data)
    
    # 3. Append the model.inertia_ to the inertia list
    inertia_values.append(kmeans.inertia_)

C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

In [158]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {"k_values": k_values, "inertia_values": inertia_values}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)


In [159]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(x="k_values", y="inertia_values", title="Elbow Curve", xlabel="Number of Clusters (k)", ylabel="Inertia", width=800, height=400)


:Curve   [k_values]   (inertia_values)

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The best value for k can be determined visually by identifying the "elbow" point on which the plotted curve. This "elbow" point represents the point where the inertia starts decreasing at a slower rate. It indicates the optimal number of clusters. From the Elbow Curve above, the k value is shown as 2. 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [160]:
# Initialize the K-Means model using the best value for k
best_k =  2

kmeans = KMeans(n_clusters=best_k, random_state=42)

In [161]:
# Fit the K-Means model using the scaled data
kmeans.fit(df_market_data)


C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=2, random_state=42)

In [162]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = kmeans.predict(df_market_data)


# Print the resulting array of cluster values.
print(clusters)


[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0]


In [163]:
# Create a copy of the DataFrame
clustered_df = df_market_data.copy()


In [164]:
# Add a new column to the DataFrame with the predicted clusters
clustered_df['Cluster'] = clusters


# Display sample data
clustered_df.head()


,price_change_percentage_24h,price_change_percentage_7d,price_change_percentage_14d,price_change_percentage_30d,price_change_percentage_60d,price_change_percentage_200d,price_change_percentage_1y,Cluster
coin_id,,,,,,,,
bitcoin,1.08388,7.60278,6.57509,7.67258,-3.25185,83.51840,37.51761,0
ethereum,0.22392,10.38134,4.80849,0.13169,-12.88890,186.77418,101.96023,0
tether,-0.21173,0.04935,0.00640,-0.04237,0.28037,-0.00542,0.01954,0
ripple,-0.37819,-0.60926,2.24984,0.23455,-17.55245,39.53888,-16.60193,0
bitcoin-cash,2.90585,17.09717,14.75334,15.74903,-13.71793,21.66042,14.49384,0


In [165]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
clustered_df.hvplot.scatter(x="price_change_percentage_24h", y="price_change_percentage_7d",
                            by="Cluster", hover_cols=["coin_id"],
                            title="Cryptocurrency Clusters", 
                            xlabel="Price Change Percentage (24h)",
                            ylabel="Price Change Percentage (7d)")

:NdOverlay   [Cluster]
   :Scatter   [price_change_percentage_24h]   (price_change_percentage_7d,coin_id)

---

### Optimize Clusters with Principal Component Analysis.

In [166]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [167]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
pca_data = pca.fit_transform(df_market_data)

# View the first five rows of the DataFrame. 
print(pca_data[:5])


[[-341.80096268  -51.36677548   12.52547089]
 [-249.42046633   24.11754777  -14.23146597]
 [-402.61472077 -118.71073742   24.83839662]
 [-406.75243715  -79.48728629    1.56633057]
 [-382.42994789 -103.43195906   16.75307273]]


In [168]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
explained_variance = pca.explained_variance_ratio_
print("Explained Variance of Principal Components:", explained_variance)

Explained Variance of Principal Components: [9.76037313e-01 2.30282949e-02 7.48308214e-04]


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is the sum of the explained variances of each principal component. It is approximately 97.60% + 2.30% + 0.75% = 100.65%. However, since explained variances represent proportions, it is common for them to sum up to slightly more than 100%. Thus, the total explained variance should be interpreted as capturing nearly all of the variability in the original data.

In [169]:
# Create a new DataFrame with the PCA data.
pca_df = pd.DataFrame(
    data=pca_data,
    columns=["PC1", "PC2", "PC3"],
    index=df_market_data.index
)

# Copy the crypto names from the original data
pca_df['coin_id'] = df_market_data.index

# Set the coinid column as index
pca_df.set_index('coin_id', inplace=True)

# Display sample data
pca_df.head()

,PC1,PC2,PC3
coin_id,,,
bitcoin,-341.800963,-51.366775,12.525471
ethereum,-249.420466,24.117548,-14.231466
tether,-402.614721,-118.710737,24.838397
ripple,-406.752437,-79.487286,1.566331
bitcoin-cash,-382.429948,-103.431959,16.753073


---

### Find the Best Value for k Using the PCA Data

In [170]:
# Create a list with the number of k-values from 1 to 11
k_values = range(1, 12)

In [171]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
    # Create a KMeans model
    kmeans = KMeans(n_clusters=k, random_state=42)
    
    # Fit the model to the PCA data
    kmeans.fit(pca_data)
    
    # Append the inertia value to the list
    inertia_values.append(kmeans.inertia_)


C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak

In [172]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {
    "k_values": k_values,
    "inertia_values": inertia_values
}
# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)

In [173]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df.hvplot.line(
    x='k_values',
    y='inertia_values',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    title='Elbow Curve for PCA Data'
)

:Curve   [k_values]   (inertia_values)

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The best value for k, based on the first elbow point in the graph, is k = 2.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** The best value for k remains the same when using the PCA data as compared to the original data at k = 2. The analysis suggests that the underlying structure of the data is well preserved even after dimensionality reduction through PCA. Therefore, in this case, both analyses lead to the same optimal value for k, which indicates that k = 2 is the best choice for clustering the data.

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [174]:
# Initialize the K-Means model using the best value for k
kmeans = KMeans(n_clusters=2, random_state=42)  



In [175]:
# Fit the K-Means model using the PCA data
kmeans.fit(pca_data)

C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\AKKem\anaconda3\envs\dev\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


KMeans(n_clusters=2, random_state=42)

In [176]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters = kmeans.predict(pca_data)

# Print the resulting array of cluster values.
print("Cluster labels:", clusters)

Cluster labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0]


In [177]:
# Create a copy of the DataFrame with the PCA data
df_with_pca = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
df_with_pca['Cluster'] = clusters


# Display sample data
df_with_pca

,PC1,PC2,PC3,Cluster
coin_id,,,,
bitcoin,-341.800963,-51.366775,12.525471,0
ethereum,-249.420466,24.117548,-14.231466,0
tether,-402.614721,-118.710737,24.838397,0
ripple,-406.752437,-79.487286,1.566331,0
bitcoin-cash,-382.429948,-103.431959,16.753073,0
binancecoin,-289.125020,12.287170,34.163848,0
chainlink,28.151408,154.987995,-73.126506,0
cardano,-174.519832,80.243493,-30.392830,0
litecoin,-406.613342,-91.783029,5.016144,0


In [179]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
# Create a scatter plot using hvPlot
scatter_plot = df_with_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",  # Color the points by the cluster labels
    hover_cols='coin_id',  # Add the cryptocurrency name (index) as hover information
    title="Scatter Plot of PCA Components (PC1 vs PC2)"
)

# Display the scatter plot
scatter_plot

:NdOverlay   [Cluster]
   :Scatter   [PC1]   (PC2,coin_id)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [181]:
# Composite plot to contrast the Elbow curves
elbow_plot_optimized = elbow_df.hvplot.line(
    x='k_values',
    y='inertia_values',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    title='Elbow Curve for Optimized Data'
)

elbow_plot_original = elbow_df.hvplot.line(
    x='k_values',
    y='inertia_values',
    xlabel='Number of Clusters (k)',
    ylabel='Inertia',
    title='Elbow Curve for Original Data'
)

elbow_composite_plot = elbow_plot_optimized * elbow_plot_original


In [183]:
# Composite plot to contrast the clusters
cluster_plot_optimized = df_with_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"],
    title='Cluster Analysis for Optimized Data'
)

cluster_plot_original = df_with_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="Cluster",
    hover_cols=["coin_id"],
    title='Cluster Analysis for Original Data'
)

cluster_composite_plot = cluster_plot_optimized + cluster_plot_original

# Display composite plots
elbow_composite_plot + cluster_composite_plot

:Layout
   .Overlay.I    :Overlay
      .Curve.I  :Curve   [k_values]   (inertia_values)
      .Curve.II :Curve   [k_values]   (inertia_values)
   .NdOverlay.I  :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2,coin_id)
   .NdOverlay.II :NdOverlay   [Cluster]
      :Scatter   [PC1]   (PC2,coin_id)

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:**  The analysis indicates that there wasn't a substantial difference between the optimized clustering (presumably with PCA) and the clustering performed using the original data. Here are some key observations:

Elbow Curve Analysis: The elbow curves for both the optimized and original data display similar trends, with a noticeable elbow point suggesting an optimal number of clusters. In this case, the optimal value for k appears to be 2.

Cluster Visualization: The scatter plots of the PCA components colored by the clusters show comparable distributions and separations of data points for both optimized and original clustering. The clusters seem to occupy similar regions in the reduced feature space, indicating consistent grouping patterns.

Interpretation: The clusters identified in both cases may represent similar underlying structures within the data. This consistency suggests that the dimensionality reduction performed by PCA did not significantly alter the clustering outcomes, indicating robustness in the clustering algorithm's performance.

Impact of Dimensionality Reduction: While PCA reduced the feature space and potentially simplified the data representation, it appears that the essential clustering patterns were preserved. This suggests that the reduced feature set captured the primary variance in the data relevant for clustering.

Practical Considerations: The similarity between optimized and original clustering outcomes implies that, in this specific case, the dimensionality reduction through PCA may not have provided significant advantages over clustering with the original data. However, this outcome may vary depending on the dataset and the specific clustering task.

In summary, while there wasn't a substantial difference between optimized clustering with PCA and clustering using the original data, the analysis suggests that both approaches yielded consistent and interpretable clustering results, with PCA potentially offering benefits in terms of dimensionality reduction and computational efficiency.
